#Instalar modulos

In [1]:
!pip install pandas==1.3.5
!pip install openpyxl==3.0.9
!pip install xlrd==2.0.1
!pip install pyreadstat

     |████████████████████████████████| 242 kB 5.2 MB/s 
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 2.5.9
    Uninstalling openpyxl-2.5.9:
      Successfully uninstalled openpyxl-2.5.9
     |████████████████████████████████| 96 kB 2.6 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
     |████████████████████████████████| 2.4 MB 5.2 MB/s 


#Importar módulos

In [2]:
import pandas as pd
import numpy as np
import os
from os.path import exists
import inspect
import datetime
import csv
import pyreadstat

#Conexión a Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Variables

In [5]:
path = "/content/drive/Shared drives/BID/Ciudades/"
path_data = path + "MicroDatos/"
path_salida = path + "Salida/Extraccion/"
file = path + "parametros.xlsx"
encoding='iso-8859-1'

# Dataframes

In [6]:
df_paises = pd.read_excel(file, sheet_name = 'paises')
#df_obj = df_paises.select_dtypes(['object'])
#df_paises[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
df_paises['pais'] = df_paises['pais'].str.strip()
df_paises['carpeta'] = df_paises['carpeta'].str.strip()
df_paises = df_paises[df_paises['procesar'] == 1]

df_ciudades = pd.read_excel(file, sheet_name = 'ciudades')
df_ciudades['pais'] = df_ciudades['pais'].str.strip()

df_sectores = pd.read_excel(file, sheet_name = 'sectores_ok')
df_sectores['pais'] = df_sectores['pais'].str.strip()
if df_sectores['cod_ciiu'].dtypes == object:
    df_sectores["cod_ciiu"] = pd.to_numeric(df_sectores["cod_ciiu"], downcast="integer", errors='coerce')
    df_sectores["cod_ciiu"] = df_sectores["cod_ciiu"].fillna(0).astype(int)
df_sectores = df_sectores[['pais', 'cod_ciiu', 'sector']]

df_archivos = pd.read_excel(file, sheet_name = 'archivos')
#df_archivos = pd.read_excel(file, sheet_name = 'archivos_pruebas')
#df_obj = df_archivos.select_dtypes(['object'])
#df_archivos[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

try:
    df_archivos["ciudad"] = df_archivos["ciudad"].str.lower()
    df_archivos["sector"] = df_archivos["sector"].str.lower()
    df_archivos["ocupado"] = df_archivos["ocupado"].str.lower()
    df_archivos["salario"] = df_archivos["salario"].str.lower()
    df_archivos["factor_exp"] = df_archivos["factor_exp"].str.lower()
except:
    pass

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# Verificar dataframes

In [7]:
df_paises
#df_sectores
#df_sectores.sector.unique()
#df_paises[df_paises['procesar']==1]
#df_archivos
#df_archivos[df_archivos['carpeta'] == 'Republica Dominicana']
#df_ciudades
#df_ciudades[['codigo_ciudad','nombre']][df_ciudades['pais'] == 'El Salvador']
#df_ciudades.loc[df_ciudades['pais'] == 'Perú', ['codigo_ciudad','nombre']]

,pais,carpeta,procesar,ok,ocupado_valor,suelo_valor,mes_valor,Observaciones,Responble ajuste,Unnamed: 9,Observaciones Daniel
0,México,Mexico,1.0,1.0,1.0,NaN,NaN,ok,W,NaN,NaN
7,Perú,Peru,1.0,1.0,1.0,NaN,NaN,ok,W,NaN,NaN


#**Procedimiento**

In [8]:
debug=0 #1=imprime todos los mensajes
pd.options.mode.chained_assignment = None #'warn'
carpeta=""
pais=""
pais_ant=""
anio_ant=""
trimestre_ant=""
archivo=""
orden_salida=0
columns_rename={}
#columns_order=['pais', 'anio', 'trimestre' ,'cod_ciudad', 'ciudad', 'cod_sector', 'sector', 'rama', 'ocupado', 'salario', 'factor_exp']
columns_order=['pais', 'anio', 'trimestre' ,'cod_ciudad', 'ciudad', 'cod_sector', 'sector', 'ocupado', 'salario', 'factor_exp']
columns_conteo = 0 
columns_conteo_check = 5

class bcolors:
    OK = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    BLUE = '\033[94m'
    ENDC = '\033[0m'

def time_dif(time1):
    time2 = datetime.datetime.now()
    td = time2 - time1
    td_mins = int(td.total_seconds()) // 60
    td_seconds = int(td.total_seconds()) % 60
    print("tiempo de ejecución:", td_mins, "m.", td_seconds, "s.", ' --- ', time2)

def print_var(p1, p2=None, p3=None, p4=None, p5=None):
    if debug==1:
        frame = inspect.currentframe()
        frame = inspect.getouterframes(frame)[1]
        string = inspect.getframeinfo(frame[0]).code_context[0].strip()
        args = string[string.find('(') + 1:-1].split(',')
        names = []
        j = 1
        for i in args:
            p = eval("p"+str(j))
            var = i + ":" + str(p)
            names.append(var)
            j+=1
        names = " ".join(str(x) for x in names)
        print(names)

def print_s(msg):
    i = 100
    j = (i - len(msg)) // 2
    if (i - len(msg)) % 2 == 0:
        k = j
    else:
        k = j + 1
    print("")
    print("#"*j, msg, "#"*k)

def read_df(file):
    df = []
    filename, file_extension = os.path.splitext(file)
    file_extension = file_extension[1:]
    file_extension = file_extension.lower()
    print_var(file)
    try:
        if file_extension == "csv":
            try:
                reader = pd.read_csv(file, sep = None, iterator = True, engine='python')
                inferred_sep = reader._engine.data.dialect.delimiter
                df = pd.read_csv(file, sep=inferred_sep, low_memory=False, encoding=encoding)
            except:
                try:    
                    df = pd.read_csv(file, sep='[,;]', quotechar='"', quoting=csv.QUOTE_NONE, engine='python', encoding=encoding) 
                except:
                    df = pd.read_csv(file, sep='[,;]', index_col=False, quotechar='"', quoting=csv.QUOTE_NONE, engine='python', encoding=encoding)
                    df.drop(df.columns[0], axis=1, inplace=True)
        elif file_extension in ("txt"):
            if os.stat(file).st_size == 0:
                print("No existen registros")
            else:
                try:
                    df = pd.read_csv(file, sep=' ', low_memory=False)
                except:
                    df = pd.read_csv(file, sep=' ', index_col=False, low_memory=False, engine='python', encoding=encoding)
                    df.drop(df.columns[0], axis=1, inplace=True)
        elif file_extension in ("xls", "xlsx"):
            if col_hoja == col_hoja:
                df = pd.read_excel(file, sheet_name=col_hoja) 
            else:
                df = pd.read_excel(file, sheet_name=0)
        elif file_extension in ("sav"):
            #df = pd.read_spss(file)
            df,meta= pyreadstat.read_sav(file)
        elif file_extension in ("dta"):
            df = pd.read_stata(file)
        else:
            print("Archivo no reconocido")
    except Exception as e:
        print("Archivo leyendo archivo: " + str(e))
    return df

def columns_df(df):
    error_columns=0
    columns=list()
    columns_error=list()
    columns_rename.clear()
    
    df.columns = df.columns.str.strip('"')
    columns_all=df.columns
    columns_all_s=", ".join(str(x) for x in columns_all)
    #print_var(columns_all)
    #print_var(columns_all_s)
    
    print_var(col_ciudad)
    print_var(df.columns)
    if col_ciudad in df.columns:
        columns.append(col_ciudad)
        columns_rename[col_ciudad] = "cod_ciudad"
    elif col_ciudad != col_ciudad:
        pass
    else:
        columns_error.append(col_ciudad)
        
    if col_ocupado in df.columns:
        columns.append(col_ocupado)
        columns_rename[col_ocupado] = "ocupado"
    elif col_ocupado != col_ocupado:
        pass
    else:
        columns_error.append(col_ocupado)
        
    if col_sector in df.columns:
        columns.append(col_sector)
        columns_rename[col_sector] = "cod_sector"
    elif col_sector != col_sector:
        pass
    else:
        columns_error.append(col_sector)
        
    if col_salario in df.columns:
        columns.append(col_salario)
        columns_rename[col_salario] = "salario"
    elif col_salario != col_salario:
        pass
    else:
        columns_error.append(col_salario)
        
    if col_factor_exp in df.columns:
        columns.append(col_factor_exp)
        columns_rename[col_factor_exp] = "factor_exp"
    elif col_factor_exp != col_factor_exp:
        pass
    else:
        columns_error.append(col_factor_exp)

    if col_llave == col_llave:
        columns.append(col_llave)
                        
    #print_var(columns)
    print_var(columns_rename)
    print_var(columns_error)
    columns_error = " ".join(str(x) for x in columns_error)
    return columns, columns_error, columns_all_s

time1=datetime.datetime.now()
cur_row = -1
for index, row in df_paises.iterrows():
    cur_row += 1
    carpeta = row['carpeta']
    pais = row['pais']
    suelo_valor = df_paises.iloc[cur_row]['suelo_valor']
    mes_valor = df_paises.iloc[cur_row]['mes_valor']
    print_s(pais)
    df_archivos_pais = df_archivos[df_archivos.carpeta == carpeta]
    if df_archivos_pais.empty == True:
        print(bcolors.BLUE, 'No existen archivos con país "', carpeta, '"', bcolors.ENDC)
    else:
        for index, row in df_archivos_pais.iterrows():
            col_archivo = row['archivo']
            col_anio = row['year']
            col_trimestre = row['trimestre']
            try:
                col_ocupado = row['ocupado'].strip()
            except:
                col_ocupado = row['ocupado']

            try:
                col_sector = row['sector'].strip()
            except:
                col_sector = row['sector']

            try:
                col_salario = row['salario'].strip()
            except:
                col_salario = row['salario']
            
            try:
                col_ciudad = row['ciudad'].strip()
            except:
                col_ciudad = row['ciudad']
            
            try:
                col_factor_exp = row['factor_exp'].strip()
            except:
                col_factor_exp = row['factor_exp']
            
            try:
                col_suelo = row['suelo']
                col_suelo = col_suelo.strip()
            except:
                col_suelo = '0'
                pass

            col_mes = row['mes']
            try:
                col_mes = col_mes.strip()
            except:
                pass

            col_hoja = row['hoja_excel']
            try:
                col_hoja = col_hoja.strip()
            except:
                pass

            col_llave = row['llave']
            try:
                col_llave = col_llave.strip()
            except:
                pass
            #columns_conteo = 0
            
            if pais == pais_ant and col_anio == anio_ant and col_trimestre == trimestre_ant:
                orden_salida+=1
            elif pais != pais_ant:
                columns_conteo = 0
                orden_salida=0
            else:
                columns_conteo = 0
                orden_salida=0

            file = path_data + str(carpeta )+ "/" + str(col_archivo)
            filename, file_extension = os.path.splitext(file)
            file_extension = file_extension[1:]
            #print_var(file)

            if exists(file):
                print("procesando... ", file)
                #try:
                df_data = read_df(file)
                #print('df_data:', len(df_data.index))
                if len(df_data) > 0:
                    #print_var(df_data)
                    df_data.columns = df_data.columns.str.lower()
                    
                    check_columns = columns_df(df_data)
                    #print('check_columns:', check_columns)
                    if len(check_columns[1]) > 0:
                        print(bcolors.BLUE, "No existen las columnas:", check_columns[1], bcolors.ENDC)
                        print(bcolors.BLUE, "columnas existentes:", check_columns[2], "\n", bcolors.ENDC)
                    else:
                        #columns = columns_df(df_data)
                        columns = check_columns[0]
                        print('columns:', columns)
                        columns_conteo += len(columns)
                        
                        #print('col_suelo:', col_suelo)
                        #print('suelo_valor:', suelo_valor)
                        if col_suelo == col_suelo:
                            #print(type(suelo_valor))
                            if type(suelo_valor) == str:
                                if ',' in suelo_valor:
                                    suelo_valor_list = suelo_valor.split(',')
                                    suelo_valor_list = [int(i) for i in suelo_valor_list]
                                    #print('suelo_valor_list:', suelo_valor_list)
                                    df_data = df_data[df_data[col_suelo].isin(suelo_valor_list)]
                                else:
                                    df_data = df_data[df_data[col_suelo] == suelo_valor]
                            else:
                                df_data = df_data[df_data[col_suelo] == suelo_valor]
                            #print('df_data:', len(df_data.index))

                        #print('col_mes:', col_mes)
                        #print('mes_valor:', mes_valor)
                        if col_mes == col_mes:
                            #print(type(mes_valor))
                            if type(mes_valor) == str:
                                if ',' in mes_valor:
                                    mes_valor_list = mes_valor.split(',')
                                    mes_valor_list = [int(i) for i in mes_valor_list]
                                    #print('mes_valor_list:', mes_valor_list)
                                    df_data = df_data[df_data[col_mes].isin(mes_valor_list)]
                                else:
                                    df_data = df_data[df_data[col_mes] == mes_valor]
                            else:
                                df_data = df_data[df_data[col_mes] == mes_valor]
                            #print('df_data:', len(df_data.index))

                        df_data = df_data[columns]
                        #print('df_data:', len(df_data.index))
                        df_data.rename(columns=dict(columns_rename), inplace=True)
                        #print('df_data:', len(df_data.index))
                        #print_var(df_data)

                        file_out=path_salida+carpeta+"_"+str(int(col_anio))+"_"+col_trimestre+".csv"
                        #file_out=path_salida+carpeta+"_"+str(int(col_anio))+"_"+col_trimestre+".xlsx"
                        if orden_salida == 0:
                            df_data.insert(loc=0, column='pais', value=pais)
                            df_data.insert(loc=1, column='anio', value=col_anio)
                            df_data.insert(loc=2, column='trimestre', value=col_trimestre)
                        else:
                            df_data2 = df_data.copy(deep=True)
                            if col_llave == col_llave:
                                df_data3 = pd.merge(df_data1, df_data2, left_on=col_llave, right_on=col_llave)
                                columns_conteo_check = 7
                            else:
                                df_data3 = pd.merge(df_data1, df_data2, left_index=True, right_index=True)
                                columns_conteo_check = 5
                            df_data = df_data3 

                        if columns_conteo == columns_conteo_check:  
                            if df_data['cod_ciudad'].dtypes == object:
                                df_data['cod_ciudad'] = df_data['cod_ciudad'].str.strip('"')
                                df_data["cod_ciudad"] = pd.to_numeric(df_data["cod_ciudad"], downcast="integer", errors='coerce')
                                df_data["cod_ciudad"] = df_data["cod_ciudad"].fillna(0).astype(int)
                            if df_data['cod_sector'].dtypes == object:
                                df_data['cod_sector'] = df_data['cod_sector'].str.strip('"')
                                df_data["cod_sector"] = pd.to_numeric(df_data["cod_sector"], downcast="integer", errors='coerce')
                                df_data["cod_sector"] = df_data["cod_sector"].fillna(0).astype(int)
                            try:
                                if df_data['ocupado'].dtypes == 'category':
                                    df_data['ocupado'] = df_data.ocupado.cat.codes
                            except:
                                pass
                            
                            if df_data['ocupado'].dtypes == object:
                                df_data['ocupado'] = df_data['ocupado'].str.strip('"')
                                df_data["ocupado"] = pd.to_numeric(df_data["ocupado"], downcast="integer", errors='coerce')
                                df_data["ocupado"] = df_data["ocupado"].fillna(0).astype(int)
                                
                            try:
                                if df_data['salario'].dtypes == 'category':
                                    df_data['salario'] = df_data.salario.cat.codes
                            except:
                                pass
                            
                            if df_data['salario'].dtypes == object:
                                df_data['salario'] = df_data['salario'].str.strip('"')
                                df_data["salario"] = pd.to_numeric(df_data["salario"], downcast="integer", errors='coerce')
                                df_data["salario"] = df_data["salario"].fillna(0).astype(int)
                                                
                            #ajusta ciudad
                            df_ciudades_sub = df_ciudades[['codigo_ciudad','nombre']][df_ciudades["pais"] == pais]
                            if df_ciudades_sub.empty:
                                print(bcolors.BLUE, "No existen ciudades para " + pais, bcolors.ENDC)
                            df_data = df_data.merge(df_ciudades_sub, how="left", left_on=["cod_ciudad"], right_on=["codigo_ciudad"])
                            df_data = df_data.rename(columns={"nombre": "ciudad"})
                            
                            #ajusta sector
                            df_sectores_sub = df_sectores[['cod_ciiu','sector']][df_sectores["pais"] == pais]
                            if df_sectores_sub.empty:
                                print(bcolors.BLUE, "No existen sectores para " + pais, bcolors.ENDC)
                            df_data = df_data.merge(df_sectores_sub, how="left", left_on=["cod_sector"], right_on=["cod_ciiu"])
                            df_data['salario'] = df_data['salario'].fillna(0)
                            df_data['cod_sector'] = df_data['cod_sector'].fillna(0)
                            try:
                                df_data = df_data.astype({'anio':'int', 'salario':'int', 'cod_sector':'int'}) 
                            except:
                                try:
                                    df_data = df_data.astype({'anio':'int'}) 
                                    df_data = df_data.astype({'cod_sector':'int'}) 
                                    df_data = df_data.astype({'cod_sector':'int'}) 
                                except:
                                    pass
                            df_data['salario'] = df_data['salario'].replace('.',',')
                            df_data['factor_exp'] = df_data['factor_exp'].replace('.',',')
                            df_data = df_data[columns_order]
                            
                            df_data.to_csv(file_out, encoding=encoding)
                            #df_data.to_excel(file_out, encoding=encoding, index=False)
                            print(bcolors.OK, "exportado: ", file_out, bcolors.ENDC)
                            columns_conteo = 0
                            columns_conteo_check = 5
                        else:
                            print(bcolors.FAIL, "no exportado:", columns_conteo, 'columnas identificadas', bcolors.ENDC)
                            df_data1 = df_data.copy(deep=True)                           

                        print_var(pais, col_anio, col_trimestre)

                        pais_ant=pais
                        anio_ant=col_anio
                        trimestre_ant=col_trimestre
                #except:
                #    print(bcolors.FAIL, "error en:", file, "\n", bcolors.ENDC)
            else:
                print(bcolors.BLUE, "No existe:", file, "\n", bcolors.ENDC)
        #print_s("fin " + pais)
print_s("fin procedimiento")
time_dif(time1)


############################################### México ###############################################
procesando...  /content/drive/Shared drives/BID/Ciudades/MicroDatos/Mexico/coe1t217.CSV
columns: ['cd_a', 'p1', 'p4a', 'fac']


KeyError: ignored

# Validaciones

In [ ]:
df_data.dtypes

pais           object
anio          float64
trimestre      object
cod_ciudad     object
ocupado       float64
cod_sector     object
salario       float64
factor_exp    float64
dtype: object

In [ ]:
file

'/content/drive/Shared drives/BID/Ciudades/MicroDatos/Chile/ene-2021-06-mjj.csv'

In [ ]:
#df_data['cod_sector'].unique()
col_suelo

'DOMINIO'

In [ ]:
df_data = df_data[df_data[col_suelo].isin(suelo_valor_list)]

KeyError: ignored

In [ ]:
print(type(col_suelo))

<class 'float'>
